In [2]:
from bs4 import BeautifulSoup, Comment
import re
import unicodedata
import pandas as pd
import numpy as np
import html5lib
import requests
from openpyxl import Workbook
from datetime import datetime
import webbrowser

In [3]:
def parse_and_trim(content, content_type):
    if content_type == 'HTML':
        soup = BeautifulSoup(content, 'html.parser')
    else:
        soup = BeautifulSoup(content, 'html.parser')

    for tag in soup.recursiveChildGenerator():
        try:
            tag.attrs = None
        except AttributeError:
            pass

    for linebreak in soup.find_all('br'):
        linebreak.extract()
    for linebreak in soup.find_all('hr'):
        linebreak.extract()

    return soup

In [ ]:
def remove_multiple_spaces(string):
    pattern = r'\s+'
    replaced_string = re.sub(pattern, ' ', string)
    return replaced_string


def find_qrt_date(content):
    qtr_date = content.find_all(text=re.compile(
        r'for\s+(the\s+)?(fiscal\s+)?year\s+ended\s+|for\s+the\s+quarter\s+ended\s+|for\s+the\s+quarterly\s+period\s+ended\s+', re.IGNORECASE))
    qtr_match = re.search(
        r'([A-Za-z]+)\s+(\d{1,2}),\s+(\d{4})', qtr_date[0].replace('\n', ''))
    if qtr_match is None:
        qtr_match = qtr_match = re.search(
            r'([A-Za-z]+) (\d{1,2}), (\d{4})', qtr_date[1])
    return remove_multiple_spaces(str(qtr_match.group()))

In [ ]:
headers = {
    'User-Agent': 'ARES CAPITAL CORP'
}
url = 'https://www.sec.gov/Archives/edgar/data/1287750/000128775019000013/arccq1-1910q.htm'
qtr_date = 'March 31, 2019'
date = qtr_date
response = requests.get(url, headers=headers)
content = parse_and_trim(response.content, 'HTML')

In [ ]:
response = requests.get(url, headers=headers)
content = parse_and_trim(response.content, 'HTML')

In [ ]:
# print(content)

In [ ]:
def extract_tables(soup_content, qtr_date):
    master_table = None
    all_tags = soup_content.find_all(True)
    print(type(all_tags))
    count = 0
    date_str = None
    for tag in content.find_all(text=re.compile('^\s*.*\s*CONSOLIDATED\s+SCHEDULE(S|)\s+OF\s+INVESTMENTS\s*.*\s*$')):
        next_line_text = tag.next.text.strip()
        regex_pattern = r'([A-Za-z]+\s+\d{1,2},\s+\d{4})'
        date_str = re.search(regex_pattern, next_line_text)
        if date_str is None:
            next_line_text = tag.find_next(text=re.compile(regex_pattern)).text
            date_str = re.search(regex_pattern, next_line_text)
            print(date_str)
        if date_str is None:
            next_line = tag.next.text.strip()
            date_str = re.search(
                regex_pattern, next_line)
            print(date_str)
        if date_str is not None:
            date_str = str(date_str.group(1))
            date_str = unicodedata.normalize('NFKD', date_str)
            print(qtr_date)
            if qtr_date.replace(',', '').strip().lower() in date_str.replace(',', '').strip().lower():
                print(qtr_date.replace(',', '').strip().lower(),
                      date_str.replace(',', '').strip().lower())
                count += 1
                print('Table found: ')
                print('Table #', count)
                html_table = tag.find_next('table')
                if master_table is None:
                    master_table = pd.read_html(
                        html_table.prettify(), skiprows=0, flavor='bs4')[0]
                    master_table = master_table.applymap(lambda x: unicodedata.normalize(
                        'NFKD', x.strip().strip(u'\u200b').replace('—', '-')) if type(x) == str else x)
                    master_table = master_table.replace(r'^\s*$', np.nan, regex=True).replace(r'^\s*\$\s*$', np.nan,
                                                                                              regex=True)
                    master_table = master_table.dropna(how='all', axis=0)
                else:
                    new_table = pd.read_html(
                        html_table.prettify(), skiprows=0, flavor='bs4')[0]
                    new_table = new_table.applymap(lambda x: unicodedata.normalize(
                        'NFKD', x.strip().strip(u'\u200b').replace('—', '-')) if type(x) == str else x)
                    new_table = new_table.replace(r'^\s*$', np.nan, regex=True).replace(r'^\s*\$\s*$', np.nan,
                                                                                        regex=True)
                    new_table = new_table.dropna(how='all', axis=0)
                    master_table = master_table.append(
                        new_table.dropna(how='all', axis=0).reset_index(
                            drop=True).drop(index=0),
                        ignore_index=True)

    master_table = master_table.applymap(
        lambda x: x.strip().strip(u'\u200b') if type(x) == str else x)
    master_table = master_table.replace(r'^\s*$', np.nan, regex=True).replace(
        r'^\s*\$\s*$', np.nan, regex=True).replace(r'^\s*\)\s*$', np.nan, regex=True)
    return master_table

In [ ]:
for tag in content.find_all(text=re.compile('^\s*.*\s*CONSOLIDATED\s+SCHEDULE(S|)\s+OF\s+INVESTMENTS\s*.*\s*$')):
    print(tag)
    regex_pattern = r'([A-Za-z]+\s+\d{1,2},\s+\d{4})'
    next_line_text = tag.find_next(text=re.compile(regex_pattern)).text
    # print(next_line_text)
    date_str = re.search(regex_pattern, next_line_text).group(1)
    print(date_str)

In [ ]:
all_tags = content.find_all(True)
all_tags

In [ ]:
for tag in content.find_all(text=re.compile('^\s*.*\s*CONSOLIDATED\s+SCHEDULE(S|)\s+OF\s+INVESTMENTS\s*.*\s*$')):
    next_line_text = tag.next.text.strip()
    regex_pattern = r'([A-Za-z]+\s+\d{1,2},\s+\d{4})'
    date_str = re.search(regex_pattern, next_line_text)
    if date_str is None:
        next_line_text = tag.find_next(text=re.compile(regex_pattern)).text
        date_str = re.search(regex_pattern, next_line_text)
        print(date_str)
    if date_str is None:
        next_line = tag.next.text.strip()
        date_str = re.search(
            regex_pattern, next_line)
        print(date_str)
    if date_str is not None:
        date_str = str(date_str.group(1))
        date_str = unicodedata.normalize('NFKD', date_str)

In [ ]:
master_table = extract_tables(content, date)

In [ ]:
# qtr_date = date
# for tag in content.find_all(text=re.compile('^.*CONSOLIDATED\s+SCHEDULE(S|)\s+OF\s+INVESTMENTS.*$')):
#     next_line_text = tag.next.text.strip()
#     regex_pattern = r'([A-Za-z]+\s+\d{1,2},\s+\d{4})'
#     date_str = re.search(regex_pattern, next_line_text).group(1)
#     print(date_str)
#     if date_str is None:
#         next_line = tag.find_next(text=re.compile(regex_pattern)).text
#         date_str = re.search(regex_pattern, next_line)
#     if date_str is None:
#         next_line = tag.next.text.strip()
#         date_str = re.search(
#             regex_pattern, next_line).group(1)
#     if date_str is not None:
#         date_str = str(date_str)
#         date_str = unicodedata.normalize('NFKD', date_str)
#         print(date_str)
#         print(qtr_date, date_str)
#         print(type(qtr_date))
#         if qtr_date.replace(',', '').strip().lower() in date_str.replace(',', '').strip().lower():
#             print(qtr_date, date_str)

In [ ]:
master_table

In [ ]:
def process_table(soi_table_df, append_str):
    soi_table_df = soi_table_df.replace(r'^\s*\$\s*$', np.nan, regex=True)
    soi_table_df = soi_table_df.dropna(how='all', axis=1)
    soi_table_df = soi_table_df.dropna(
        how='all', axis=0).reset_index(drop=True)
    print('1: ' + str(soi_table_df.shape))

    # Separate header and data
    soi_table_header = soi_table_df.iloc[0].dropna(how='any')
    print('header: ')
    print(soi_table_header)
    soi_table_data_df = soi_table_df.rename(
        columns=soi_table_df.iloc[0]).drop(soi_table_df.index[0])
    print('2: ' + str(soi_table_data_df.shape))

    # Drop rows with only two non-null values
    soi_table_data_df = soi_table_data_df.dropna(thresh=3)
    print('4: ' + str(soi_table_data_df.shape))

    columns_to_fill = ['Amortized Cost', 'Fair Value']
    for col in columns_to_fill:
        original_column_names = soi_table_data_df.columns.tolist()
        soi_table_data_df.columns = soi_table_data_df.columns.str.replace(
            ' ', '')
        col_index = soi_table_data_df.columns.get_loc(col.replace(' ', ''))
        soi_table_data_df.columns = original_column_names
        next_col_index = col_index + 1
        for i in range(len(soi_table_data_df)):
            current_value = soi_table_data_df.iat[i, col_index]
            if pd.isna(current_value) and next_col_index < len(soi_table_data_df.columns):
                next_valid_index = next((j for j, v in enumerate(
                    soi_table_data_df.iloc[i, next_col_index:], start=next_col_index) if pd.notna(v)), None)

                if next_valid_index is not None:
                    next_value = soi_table_data_df.iat[i, next_valid_index]
                    soi_table_data_df.iat[i, col_index] = next_value
                    soi_table_data_df.iat[i, next_valid_index] = pd.NA

    # # Drop rows labeled as subtotals
    # subtotal_rows = soi_table_data_df[soi_table_data_df['Company (1)'].str.contains(
    #     'subtotal', case=False, na=False)]
    # soi_table_data_df = soi_table_data_df[~soi_table_data_df.index.isin(
    #     subtotal_rows.index)]
    # print('3: ' + str(soi_table_data_df.shape))

    # # Drop rows based on regex pattern (e.g., 'subtotal' or 'total')
    # pattern = r'^([Ss]ubtotal)|([Tt]otal)'
    # mask = soi_table_data_df.apply(lambda row: row.astype(
    #     str).str.contains(pattern, case=False, na=False)).any(axis=1)
    # soi_table_data_df = soi_table_data_df[~mask]
    # print('4: ' + str(soi_table_data_df.shape))

    # # Drop rows with all missing values
    # soi_table_df = soi_table_df.dropna(how='all')
    # print('5: ' + str(soi_table_data_df.shape))

    # # # Drop columns with all missing values
    # soi_table_data_df = soi_table_data_df.dropna(how='all', axis=1)
    # print('6: ' + str(soi_table_data_df.shape))

    # # Forward fill the first two columns
    # col_indices = [0, 1]
    # soi_table_data_df.iloc[:, col_indices] = soi_table_data_df.iloc[:, col_indices].fillna(
    #     method='ffill')
    # print('7: ' + str(soi_table_data_df.shape))

    # num_cols = soi_table_data_df.shape[1]
    # for col_index in range(num_cols-3, num_cols):
    #     col_name = soi_table_data_df.columns[col_index]
    #     soi_table_data_df[col_name] = pd.to_numeric(
    #         soi_table_data_df[col_name], errors='coerce').fillna(0)

    # soi_table_data_df = soi_table_data_df.replace('-', 0, regex=False)
    # print('8: ' + str(soi_table_data_df.shape))
    soi_table_data_df = soi_table_data_df.reset_index(drop=True)

    return soi_table_data_df


process_table_ = process_table(master_table, "hi")
process_table_.to_excel("ex.xlsx")
process_table_.to_csv('ex.csv')
process_table_

In [ ]:
# Forward fill the first two columns

In [ ]:
path = '/Users/fuadhassan/Desktop/BDC_RA/ARCC/ARCC_Investment.xlsx'
xls = pd.ExcelFile(path)
sheet_names = xls.sheet_names

# Print the sheet names
print(sheet_names)

In [ ]:
dfs = {}

# Read each sheet and store in the dictionary
for sheet_name in sheet_names:
    df = pd.read_excel(xls, sheet_name)
    dfs[sheet_name] = df

# Access and print individual DataFrames
for sheet_name, df in dfs.items():
    print(f"Sheet Name: {sheet_name}")

In [ ]:
content

In [ ]:
all_tags = content.find_all("table")

In [ ]:
for tag in content.find_all(text=re.compile('^\s*.*\s*CONSOLIDATED\s+SCHEDULE(S|)\s+OF\s+INVESTMENTS\s*.*\s*$')):
    next_line_text = tag.next.text.strip()
    regex_pattern = r'([A-Za-z]+\s+\d{1,2},\s+\d{4})'
    date_str = re.search(regex_pattern, next_line_text)
    if date_str is None:
        next_line_text = tag.find_next(text=re.compile(regex_pattern)).text
        date_str = re.search(regex_pattern, next_line_text)
        print(date_str)
    if date_str is None:
        next_line = tag.next.text.strip()
        date_str = re.search(
            regex_pattern, next_line)
        print(date_str)
    if date_str is not None:
        date_str = str(date_str.group(1))
        date_str = unicodedata.normalize('NFKD', date_str)

In [ ]:

count = 0
for tag in content.find_all(text=re.compile('^\s*.*\s*CONSOLIDATED\s+SCHEDULE(S|)\s+OF\s+INVESTMENTS\s*.*\s*$')):
    regex_pattern = r'([A-Za-z]+\s+\d{1,2},\s+\d{4})'
    next_line_text = tag.find_next(text=re.compile(regex_pattern)).text
    date_str = re.search(regex_pattern, next_line_text).group(1)
    date_str = unicodedata.normalize('NFKD', date_str)
    if qtr_date.replace(',', '').strip().lower() in date_str.replace(',', '').strip().lower():
        html_table = tag.find_next('table')
        while html_table and html_table.find_next_sibling() != None:
            new_table = pd.read_html(
                html_table.prettify(), skiprows=0, flavor='bs4')[0]
            new_table = new_table.applymap(lambda x: unicodedata.normalize(
                'NFKD', x.strip().strip(u'\u200b').replace('—', '-')) if type(x) == str else x)
            new_table = new_table.replace(
                r'^\s*$', np.nan, regex=True).replace(r'^\s*\$\s*$', np.nan, regex=True)
            new_table = new_table.dropna(how='all', axis=0)

            if master_table is None:
                master_table = new_table
            else:
                master_table = pd.concat(
                    [master_table, new_table], ignore_index=True)
            html_table = html_table.find_next('table')
master_table = master_table.applymap(
    lambda x: x.strip().strip(u'\u200b') if type(x) == str else x)
master_table = master_table.replace(r'^\s*$', np.nan, regex=True).replace(
    r'^\s*\$\s*$', np.nan, regex=True).replace(r'^\s*\)\s*$', np.nan, regex=True)

In [ ]:
master_table

In [ ]:

count = 0
for tag in content.find_all(text=re.compile('^\s*.*\s*CONSOLIDATED\s+SCHEDULE(S|)\s+OF\s+INVESTMENTS\s*.*\s*$')):
    regex_pattern = r'([A-Za-z]+\s+\d{1,2},\s+\d{4})'
    next_line_text = tag.find_next(text=re.compile(regex_pattern)).text
    date_str = re.search(regex_pattern, next_line_text).group(1)
    date_str = unicodedata.normalize('NFKD', date_str)
    if qtr_date.replace(',', '').strip().lower() in date_str.replace(',', '').strip().lower():
        html_table = tag.find_next('table')
        while html_table:
            if html_table.find_next(text=re.compile('^\s*.*\s*CONSOLIDATED\s+SCHEDULE(S|)\s+OF\s+INVESTMENTS\s*.*\s*$')):
                print(html_table.find_next(text=re.compile(
                    '^\s*.*\s*CONSOLIDATED\s+SCHEDULE(S|)\s+OF\s+INVESTMENTS\s*.*\s*$')))
                break
            html_table = html_table.find_next('table')

In [ ]:
for tag in content.find_all(text=re.compile(r'\s+Pledged\s', re.IGNORECASE)):
    # Process the tag that matches the pattern
    print(tag)

In [ ]:
with open('content.txt', 'w') as file:
    file.write(str(content))

In [ ]:

def extract_tables(soup_content, qtr_date):
    master_table = None
    all_tags = soup_content.find_all(True)
    count = 0
    for tag in content.find_all(text=re.compile('^\s*.*\s*CONSOLIDATED\s+SCHEDULE(S|)\s+OF\s+INVESTMENTS\s*.*\s*$')):
        regex_pattern = r'([A-Za-z]+\s+\d{1,2},\s+\d{4})'
        next_line_text = tag.find_next(text=re.compile(regex_pattern)).text
        date_str = re.search(regex_pattern, next_line_text).group(1)
        date_str = unicodedata.normalize('NFKD', date_str)
        if qtr_date.replace(',', '').strip().lower() in date_str.replace(',', '').strip().lower():
            html_table = tag.find_next('table')
            while html_table:
                if html_table.find_next(text=re.compile('^\s*.*\s*CONSOLIDATED\s+SCHEDULE(S|)\s+OF\s+INVESTMENTS\s*.*\s*$')):
                    break
                new_table = pd.read_html(
                    html_table.prettify(), skiprows=0, flavor='bs4')[0]
                new_table = new_table.applymap(lambda x: unicodedata.normalize(
                    'NFKD', x.strip().strip(u'\u200b').replace('—', '-')) if type(x) == str else x)
                new_table = new_table.replace(
                    r'^\s*$', np.nan, regex=True).replace(r'^\s*\$\s*$', np.nan, regex=True)
                new_table = new_table.dropna(how='all', axis=0)

                if master_table is None:
                    master_table = new_table
                else:
                    master_table = pd.concat(
                        [master_table, new_table], ignore_index=True)
                html_table = html_table.find_next('table')
    master_table = master_table.applymap(
        lambda x: x.strip().strip(u'\u200b') if type(x) == str else x)
    master_table = master_table.replace(r'^\s*$', np.nan, regex=True).replace(
        r'^\s*\$\s*$', np.nan, regex=True).replace(r'^\s*\)\s*$', np.nan, regex=True)

    return master_table


master_table = extract_tables(content, qtr_date)
master_table

In [ ]:
def extract_tables(soup_content, qtr_date):
    master_table = None
    all_tags = soup_content.find_all(True)
    count = 0
    for tag in soup_content.find_all(text=re.compile('^\s*.*\s*CONSOLIDATED\s+SCHEDULE(S|)\s+OF\s+INVESTMENTS\s*.*\s*$')):
        next_line_text = tag.next.text.strip()
        regex_pattern = r'([A-Za-z]+\s+\d{1,2},\s+\d{4})'
        date_str = re.search(regex_pattern, next_line_text)
        if date_str is None:
            next_line_text = tag.find_next(text=re.compile(regex_pattern)).text
            date_str = re.search(regex_pattern, next_line_text)
        if date_str is None:
            next_line = tag.next.text.strip()
            date_str = re.search(
                regex_pattern, next_line)
        if date_str is not None:
            date_str = str(date_str.group(1))
            date_str = unicodedata.normalize('NFKD', date_str)
            if qtr_date.replace(',', '').strip().lower() in date_str.replace(',', '').strip().lower():
                count += 1
                # print('Table #', count)
                html_table = tag.find_next('table')
                while html_table and html_table.find_next(text=re.compile('^\s*.*\s*CONSOLIDATED\s+SCHEDULE(S|)\s+OF\s+INVESTMENTS\s*.*\s*$')):
                    new_table = pd.read_html(
                        html_table.prettify(), skiprows=0, flavor='bs4')[0]
                    new_table = new_table.applymap(lambda x: unicodedata.normalize(
                        'NFKD', x.strip().strip(u'\u200b').replace('—', '-')) if type(x) == str else x)
                    new_table = new_table.replace(
                        r'^\s*$', np.nan, regex=True).replace(r'^\s*\$\s*$', np.nan, regex=True)
                    new_table = new_table.dropna(how='all', axis=0)

                    if master_table is None:
                        master_table = new_table
                    else:
                        master_table = pd.concat(
                            [master_table, new_table], ignore_index=True)
                    next_schedule_tag = html_table.find_next(text=re.compile(
                        '^\s*.*\s*CONSOLIDATED\s+SCHEDULE(S|)\s+OF\s+INVESTMENTS\s*.*\s*$'))
                    if not next_schedule_tag:
                        break
                    html_table = html_table.find_next('table')

    master_table = master_table.applymap(
        lambda x: x.strip().strip(u'\u200b') if type(x) == str else x)
    master_table = master_table.replace(r'^\s*$', np.nan, regex=True).replace(
        r'^\s*\$\s*$', np.nan, regex=True).replace(r'^\s*\)\s*$', np.nan, regex=True)
    return master_table


master_table = extract_tables(content, qtr_date)
master_table

In [ ]:
        lambda x: x.strip().strip(u'\u200b') if type(x) == str else x)
    master_table = master_table.replace(r'^\s*$', np.nan, regex=True).replace(
        r'^\s*\$\s*$', np.nan, regex=True).replace(r'^\s*\)\s*$', np.nan, regex=True)
    return master_table


master_table = extract_tables(content, qtr_date)
master_table

In [ ]:
process_table_ = process_table(master_table, "")
process_table_.to_excel("example.xlsx")
process_table_.to_csv('example.csv')
process_table_

In [ ]:

count = 0
for tag in content.find_all(text=re.compile('^\s*.*\s*CONSOLIDATED\s+SCHEDULE(S|)\s+OF\s+INVESTMENTS\s*.*\s*$')):
    regex_pattern = r'([A-Za-z]+\s+\d{1,2},\s+\d{4})'
    next_line_text = tag.find_next(text=re.compile(regex_pattern)).text
    date_str = re.search(regex_pattern, next_line_text).group(1)
    date_str = unicodedata.normalize('NFKD', date_str)
    if qtr_date.replace(',', '').strip().lower() in date_str.replace(',', '').strip().lower():
        html_table = tag.find_next('table')
        while html_table and html_table.find_next(text=re.compile('^\s*.*\s*CONSOLIDATED\s+SCHEDULE(S|)\s+OF\s+INVESTMENTS\s*.*\s*$')):
            html_table = html_table.find_next('table')

In [ ]:
def extract_tables(soup_content, qtr_date):
    master_table = None
    all_tags = soup_content.find_all(True)
    count = 0
    for tag in soup_content.find_all(text=re.compile('^\s*.*\s*CONSOLIDATED\s+SCHEDULE(S|)\s+OF\s+INVESTMENTS\s*.*\s*$')):
        next_line_text = tag.next.text.strip()
        regex_pattern = r'([A-Za-z]+\s+\d{1,2},\s+\d{4})'
        date_str = re.search(regex_pattern, next_line_text)
        if date_str is None:
            next_line_text = tag.find_next(text=re.compile(regex_pattern)).text
            date_str = re.search(regex_pattern, next_line_text)
        if date_str is None:
            next_line = tag.next.text.strip()
            date_str = re.search(
                regex_pattern, next_line)
        if date_str is not None:
            date_str = str(date_str.group(1))
            date_str = unicodedata.normalize('NFKD', date_str)
            if qtr_date.replace(',', '').strip().lower() in date_str.replace(',', '').strip().lower():
                html_table = tag.find_next('table')

In [ ]:
def extract_tables(soup_content, qtr_date):
    consolidated_schedule_regex = re.compile(
        r'^\s*.*\s*CONSOLIDATED\s+SCHEDULE(S|)\s+OF\s+INVESTMENTS\s*.*\s*$')
    date_regex_pattern = r'([A-Za-z]+\s+\d{1,2},\s+\d{4})'

    master_table = None
    all_tags = soup_content.find_all(True)
    count = 0

    for tag in soup_content.find_all(text=consolidated_schedule_regex):
        next_line_text = tag.next.text.strip()
        date_str = re.search(date_regex_pattern, next_line_text)

        if date_str is None:
            next_line_text = tag.find_next(
                text=re.compile(date_regex_pattern)).text
            date_str = re.search(date_regex_pattern, next_line_text)

        if date_str is None:
            next_line = tag.next.text.strip()
            date_str = re.search(date_regex_pattern, next_line)

        if date_str is not None:
            date_str = str(date_str.group(1))
            date_str = unicodedata.normalize('NFKD', date_str)

            if qtr_date.replace(',', '').strip().lower() in date_str.replace(',', '').strip().lower():
                count += 1
                html_table = tag.find_next('table')
                while html_table and html_table.find_next(text=consolidated_schedule_regex):
                    new_table = pd.read_html(
                        html_table.prettify(), skiprows=0, flavor='bs4')[0]
                    new_table = new_table.applymap(lambda x: unicodedata.normalize(
                        'NFKD', x.strip().strip(u'\u200b').replace('—', '-')) if type(x) == str else x)
                    new_table = new_table.replace(
                        r'^\s*$', np.nan, regex=True).replace(r'^\s*\$\s*$', np.nan, regex=True)
                    new_table = new_table.dropna(how='all', axis=0)

                    if master_table is None:
                        master_table = new_table
                    else:
                        master_table = pd.concat(
                            [master_table, new_table], ignore_index=True)

                    html_table = html_table.find_next('table')

    master_table = master_table.applymap(
        lambda x: x.strip().strip(u'\u200b') if type(x) == str else x)
    master_table = master_table.replace(r'^\s*$', np.nan, regex=True).replace(
        r'^\s*\$\s*$', np.nan, regex=True).replace(r'^\s*\)\s*$', np.nan, regex=True)
    return master_table


master_table = extract_tables(content, qtr_date)
master_table

In [ ]:
headers = {
    'User-Agent': 'ARES CAPITAL CORP'
}
url = 'https://www.sec.gov/Archives/edgar/data/1287750/000128775019000013/arccq1-1910q.htm'
qtr_date = 'March 31, 2019'
# url = 'https://www.sec.gov/Archives/edgar/data/1287750/000104746911001575/a2202281z10-k.htm'
# qtr_date = 'December 31, 2010'

response = requests.get(url, headers=headers)
content = parse_and_trim(response.content, 'HTML')


# def extract_tables(soup_content, qtr_date):
#     consolidated_schedule_regex = re.compile(
#         r'^\s*.*\s*CONSOLIDATED\s+SCHEDULE(S|)\s+OF\s+INVESTMENTS\s*.*\s*$')
#     date_regex_pattern = r'([A-Za-z]+\s+\d{1,2},\s+\d{4})'

#     master_table = None
#     all_tags = soup_content.find_all(True)
#     count = 0

#     for tag in soup_content.find_all(text=consolidated_schedule_regex):
#         next_line_text = tag.next.text.strip()
#         date_str = re.search(date_regex_pattern, next_line_text)

#         if date_str is None:
#             next_line_text = tag.find_next(
#                 text=re.compile(date_regex_pattern)).text
#             date_str = re.search(date_regex_pattern, next_line_text)

#         if date_str is None:
#             next_line = tag.next.text.strip()
#             date_str = re.search(date_regex_pattern, next_line)

#         if date_str is not None:
#             date_str = str(date_str.group(1))
#             date_str = unicodedata.normalize('NFKD', date_str)

#             if qtr_date.replace(',', '').strip().lower() in date_str.replace(',', '').strip().lower():
#                 count += 1
#                 html_table = tag.find_next('table')
#                 while html_table and html_table.find_next(text=consolidated_schedule_regex):
#                     new_table = pd.read_html(
#                         html_table.prettify(), skiprows=0, flavor='bs4')[0]
#                     new_table = new_table.applymap(lambda x: unicodedata.normalize(
#                         'NFKD', x.strip().strip(u'\u200b').replace('—', '-')) if type(x) == str else x)
#                     new_table = new_table.replace(
#                         r'^\s*$', np.nan, regex=True).replace(r'^\s*\$\s*$', np.nan, regex=True)
#                     new_table = new_table.dropna(how='all', axis=0)

#                     if master_table is None:
#                         master_table = new_table
#                     else:
#                         master_table = pd.concat(
#                             [master_table, new_table], ignore_index=True)

#                     html_table = html_table.find_next('table')

#     master_table = master_table.applymap(
#         lambda x: x.strip().strip(u'\u200b') if type(x) == str else x)
#     master_table = master_table.replace(r'^\s*$', np.nan, regex=True).replace(
#         r'^\s*\$\s*$', np.nan, regex=True).replace(r'^\s*\)\s*$', np.nan, regex=True)
#     return master_table


# master_table = extract_tables(content, qtr_date)
# master_table

In [ ]:
headers = {
    'User-Agent': 'ARES CAPITAL CORP'
}
# url = 'https://www.sec.gov/Archives/edgar/data/1287750/000128775019000013/arccq1-1910q.htm'
# qtr_date = 'March 31, 2019'
# url = 'https://www.sec.gov/Archives/edgar/data/1287750/000104746911001575/a2202281z10-k.htm'
# qtr_date = 'December 31, 2010'
url = 'https://www.sec.gov/Archives/edgar/data/1287750/000110465910055721/a10-17362_110q.htm'
qtr_date = 'September 30, 2010'

response = requests.get(url, headers=headers)
content = parse_and_trim(response.content, 'HTML')


def extract_tables(soup_content, qtr_date):
    consolidated_schedule_regex = re.compile(
        r'^\s*.*\s*CONSOLIDATED\s+SCHEDULE(S|)\s+OF\s+INVESTMENTS\s*.*\s*$')
    date_regex_pattern = r'([A-Za-z]+\s+\d{1,2},\s+\d{4})'

    master_table = None
    all_tags = soup_content.find_all(True)
    count = 0

    for tag in soup_content.find_all(text=consolidated_schedule_regex):
        next_line_text = tag.next.text.strip()
        date_str = re.search(date_regex_pattern, next_line_text)

        if date_str is None:
            next_line_text = tag.find_next(
                text=re.compile(date_regex_pattern)).text
            date_str = re.search(date_regex_pattern, next_line_text)

        if date_str is None:
            next_line = tag.next.text.strip()
            date_str = re.search(date_regex_pattern, next_line)

        if date_str is not None:
            date_str = str(date_str.group(1))
            date_str = unicodedata.normalize('NFKD', date_str)

            if qtr_date.replace(',', '').strip().lower() in date_str.replace(',', '').strip().lower():
                count += 1
                html_table = tag.find_next('table')
                while html_table:
                    new_table = pd.read_html(
                        html_table.prettify(), skiprows=0, flavor='bs4')[0]
                    new_table = new_table.applymap(lambda x: unicodedata.normalize(
                        'NFKD', x.strip().strip(u'\u200b').replace('—', '-')) if type(x) == str else x)
                    new_table = new_table.replace(
                        r'^\s*$', np.nan, regex=True).replace(r'^\s*\$\s*$', np.nan, regex=True)
                    new_table = new_table.dropna(how='all', axis=0)

                    if master_table is None:
                        master_table = new_table
                    else:
                        master_table = pd.concat(
                            [master_table, new_table], ignore_index=True)

                    # print(master_table)
                    if html_table.find_next(text=consolidated_schedule_regex):
                        next_line_text = tag.find_next(
                            text=re.compile(date_regex_pattern)).text
                        date_str2 = re.search(
                            date_regex_pattern, next_line_text)
                        print(date_str)
                        print(date_str2.group(1))
                        if date_str2.group(1) == date_str:
                            break
                    html_table = html_table.find_next('table')

    master_table = master_table.applymap(
        lambda x: x.strip().strip(u'\u200b') if type(x) == str else x)
    master_table = master_table.replace(r'^\s*$', np.nan, regex=True).replace(
        r'^\s*\$\s*$', np.nan, regex=True).replace(r'^\s*\)\s*$', np.nan, regex=True)
    return master_table


master_table = extract_tables(content, qtr_date)
master_table

In [ ]:
headers = {
    'User-Agent': 'ARES CAPITAL CORP'
}
url = 'https://www.sec.gov/Archives/edgar/data/1287750/000128775019000013/arccq1-1910q.htm'
qtr_date = 'March 31, 2019'
url = 'https://www.sec.gov/Archives/edgar/data/1287750/000104746911001575/a2202281z10-k.htm'
qtr_date = 'December 31, 2010'
# url = 'https://www.sec.gov/Archives/edgar/data/1287750/000110465910055721/a10-17362_110q.htm'
# qtr_date = 'September 30, 2010'
# url = 'https://www.sec.gov/Archives/edgar/data/1287750/000128775018000015/arccq1-1810q.htm'
# qtr_date = 'March 31, 2018'
response = requests.get(url, headers=headers)
content = parse_and_trim(response.content, 'HTML')


def extract_tables(soup_content, qtr_date):
    consolidated_schedule_regex = re.compile(
        r'^\s*.*\s*CONSOLIDATED\s+SCHEDULE(S|)\s+OF\s+INVESTMENTS\s*.*\s*$')
    date_regex_pattern = r'([A-Za-z]+\s+\d{1,2},\s+\d{4})'

    master_table = None
    all_tags = soup_content.find_all(True)
    count = 0

    for tag in soup_content.find_all(text=consolidated_schedule_regex):
        next_line_text = tag.next.text.strip()
        date_str = re.search(date_regex_pattern, next_line_text)

        if date_str is None:
            next_line_text = tag.find_next(
                text=re.compile(date_regex_pattern)).text
            date_str = re.search(date_regex_pattern, next_line_text)

        if date_str is None:
            next_line = tag.next.text.strip()
            date_str = re.search(date_regex_pattern, next_line)

        if date_str is not None:
            date_str = str(date_str.group(1))
            date_str = unicodedata.normalize('NFKD', date_str)

            if qtr_date.replace(',', '').strip().lower() in date_str.replace(',', '').strip().lower():
                count += 1
                html_table = tag.find_next('table')
                while html_table and html_table.find_next(text=consolidated_schedule_regex):
                    print('table found')
                    # if html_table.find_next(text=consolidated_schedule_regex):
                    #     print('next text found')
                    next_line_text = tag.find_next(
                        text=re.compile(date_regex_pattern)).text
                    date_str2 = re.search(
                        date_regex_pattern, next_line_text)
                    print(date_str2.group(1), date_str)
                    print(date_str2.group(1) is not date_str)
                    if date_str2.group(1) is not date_str:
                        print("broke here")
                        break
                    print("moved to next table")
                    html_table = html_table.find_next('table')

    # master_table = master_table.applymap(
    #     lambda x: x.strip().strip(u'\u200b') if type(x) == str else x)
    # master_table = master_table.replace(r'^\s*$', np.nan, regex=True).replace(
    #     r'^\s*\$\s*$', np.nan, regex=True).replace(r'^\s*\)\s*$', np.nan, regex=True)
    # return master_table


master_table = extract_tables(content, qtr_date)
master_table

In [ ]:
headers = {
    'User-Agent': 'ARES CAPITAL CORP'
}
url = 'https://www.sec.gov/Archives/edgar/data/1287750/000128775019000013/arccq1-1910q.htm'
qtr_date = 'March 31, 2019'
# url = 'https://www.sec.gov/Archives/edgar/data/1287750/000104746911001575/a2202281z10-k.htm'
# qtr_date = 'December 31, 2010'
# url = 'https://www.sec.gov/Archives/edgar/data/1287750/000110465910055721/a10-17362_110q.htm'
# qtr_date = 'September 30, 2010'
# url = 'https://www.sec.gov/Archives/edgar/data/1287750/000128775018000015/arccq1-1810q.htm'
# qtr_date = 'March 31, 2018'
response = requests.get(url, headers=headers)
content = parse_and_trim(response.content, 'HTML')


def extract_tables(soup_content, qtr_date):
    consolidated_schedule_regex = re.compile(
        r'^\s*.*\s*CONSOLIDATED\s+SCHEDULE(S|)\s+OF\s+INVESTMENTS\s*.*\s*$')
    date_regex_pattern = r'([A-Za-z]+\s+\d{1,2},\s+\d{4})'

    master_table = None
    all_tags = soup_content.find_all(True)
    count = 0

    for tag in soup_content.find_all(text=consolidated_schedule_regex):
        next_line_text = tag.next.text.strip()
        date_str = re.search(date_regex_pattern, next_line_text)

        if date_str is None:
            next_line_text = tag.find_next(
                text=re.compile(date_regex_pattern)).text
            date_str = re.search(date_regex_pattern, next_line_text)

        if date_str is None:
            next_line = tag.next.text.strip()
            date_str = re.search(date_regex_pattern, next_line)

        if date_str is not None:
            date_str = str(date_str.group(1))
            date_str = unicodedata.normalize('NFKD', date_str)

            if qtr_date.replace(',', '').strip().lower() in date_str.replace(',', '').strip().lower():
                count += 1
                html_table = tag.find_next('table')
                while html_table and check_break_point(content):
                    print('table found')
                    # # if html_table.find_next(text=consolidated_schedule_regex):
                    # #     print('next text found')
                    # next_line_text = tag.find_next(
                    #     text=re.compile(date_regex_pattern)).text
                    # date_str2 = re.search(
                    #     date_regex_pattern, next_line_text)
                    # print(date_str2.group(1), date_str)
                    # print(date_str2.group(1) is not date_str)
                    # if date_str2.group(1) is not date_str:
                    #     print("broke here")
                    #     break
                    new_table = pd.read_html(
                        html_table.prettify(), skiprows=0, flavor='bs4')[0]
                    new_table = new_table.applymap(lambda x: unicodedata.normalize(
                        'NFKD', x.strip().strip(u'\u200b').replace('—', '-')) if type(x) == str else x)
                    new_table = new_table.replace(
                        r'^\s*$', np.nan, regex=True).replace(r'^\s*\$\s*$', np.nan, regex=True)
                    new_table = new_table.dropna(how='all', axis=0)

                    if master_table is None:
                        master_table = new_table
                    else:
                        master_table = pd.concat(
                            [master_table, new_table], ignore_index=True)
                    print("moved to next table")
                    html_table = html_table.find_next('table')

    master_table = master_table.applymap(
        lambda x: x.strip().strip(u'\u200b') if type(x) == str else x)
    master_table = master_table.replace(r'^\s*$', np.nan, regex=True).replace(
        r'^\s*\$\s*$', np.nan, regex=True).replace(r'^\s*\)\s*$', np.nan, regex=True)
    return master_table


master_table = extract_tables(content, qtr_date)
master_table

In [ ]:
def check_break_point(content):
    next_sibling = content.find_next_sibling()
    if next_sibling and "Derivative Instruments" in next_sibling.text:
        return False
    return True

In [ ]:
def find_text_in_larger_text(larger_text, target_text):
    pattern = re.escape(target_text)
    match = re.search(pattern, larger_text)
    if match:
        return match.group()
    else:
        return None

In [19]:
headers = {
    'User-Agent': 'ARES CAPITAL CORP'
}
# url = 'https://www.sec.gov/Archives/edgar/data/1287750/000128775023000036/arcc-20230630.htm'
# date = 'June 30, 2023'
# url = 'https://www.sec.gov/Archives/edgar/data/1287750/000128775019000013/arccq1-1910q.htm'
# date = 'March 31, 2019'
url = 'https://www.sec.gov/Archives/edgar/data/1287750/000104746914001349/a2218339z10-k.htm'
date = 'December 31, 2013'
# url = 'https://www.sec.gov/Archives/edgar/data/1287750/000128775018000015/arccq1-1810q.htm'
# date = 'March 31, 2018'
# url = 'https://www.sec.gov/Archives/edgar/data/1287750/000110465913080832/a13-19678_110q.htm'
# date = 'September 30, 2013'
# url = 'https://www.sec.gov/Archives/edgar/data/1287750/000128775018000007/arccq4-1710k.htm'
# date = 'December 31, 2017'
url = 'https://www.sec.gov/Archives/edgar/data/1287750/000104746915001240/a2222984z10-k.htm'
date = 'December 31, 2014'
url = 'https://www.sec.gov/Archives/edgar/data/1287750/000104746916010353/a2227293z10-k.htm'
date = 'December 31, 2015'

qtr_date = date

response = requests.get(url, headers=headers)
content = parse_and_trim(response.content, 'HTML')


def extract_tables_manual(soup_content, qtr_date):
    consolidated_schedule_regex = re.compile(
        r'^\s*.*\s*CONSOLIDATED\s+SCHEDULE(S|)\s+OF\s+INVESTMENTS\s*.*\s*$')
    date_regex_pattern = r'([A-Za-z]+\s+\d{1,2},\s+\d{4})'

    master_table = None
    all_tags = soup_content.find_all(True)
    count = 0

    for tag in soup_content.find_all(text=consolidated_schedule_regex):
        # print(len(soup_content.find_all(text=consolidated_schedule_regex)))
        next_line_text = tag.next.text.strip()
        date_str = re.search(date_regex_pattern, next_line_text)

        if date_str is None:
            next_line_text = tag.find_next(
                text=re.compile(date_regex_pattern)).text
            date_str = re.search(date_regex_pattern, next_line_text)

        if date_str is None:
            next_line = tag.next.text.strip()
            date_str = re.search(date_regex_pattern, next_line)

        if date_str is not None:
            date_str = str(date_str.group(1))
            date_str = unicodedata.normalize('NFKD', date_str)

            if qtr_date.replace(',', '').strip().lower() in date_str.replace(',', '').strip().lower():
                count += 1
                html_table = tag.find_next('table')
                while html_table:
                    new_table = pd.read_html(
                        html_table.prettify(), skiprows=0, flavor='bs4')[0]
                    new_table = new_table.applymap(lambda x: unicodedata.normalize(
                        'NFKD', x.strip().strip(u'\u200b').replace('—', '-')) if type(x) == str else x)
                    new_table = new_table.replace(
                        r'^\s*$', np.nan, regex=True).replace(r'^\s*\$\s*$', np.nan, regex=True)
                    new_table = new_table.dropna(how='all', axis=0)

                    if master_table is None:
                        master_table = new_table
                    else:
                        master_table = pd.concat(
                            [master_table, new_table], ignore_index=True)

                    if date_str.replace(',', '').strip().lower() in 'December 31, 2013'.replace(',', '').strip().lower() or date_str.replace(',', '').strip().lower() in 'December 31, 2014'.replace(',', '').strip().lower():
                        if html_table.find(text=re.compile(r'Food and Beverage', re.IGNORECASE)):
                            break
                    if date_str.replace(',', '').strip().lower() in 'December 31, 2015'.replace(',', '').strip().lower() or date_str.replace(',', '').strip().lower() in 'December 31, 2016'.replace(',', '').strip().lower():
                        if html_table.find(text=re.compile(r'Computers and Electronics', re.IGNORECASE)):
                            break

                    html_table = html_table.find_next('table')

    master_table = master_table.applymap(
        lambda x: x.strip().strip(u'\u200b') if type(x) == str else x)
    master_table = master_table.replace(r'^\s*$', np.nan, regex=True).replace(
        r'^\s*\$\s*$', np.nan, regex=True).replace(r'^\s*\)\s*$', np.nan, regex=True)
    print(master_table.shape)
    return master_table


master_table = extract_tables_manual(content, qtr_date)
master_table.to_csv('example.csv')

master_table

(647, 20)


,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19
0,Company(1),NaN,Business Description,NaN,Investment,NaN,Interest(6)(12),NaN,Acquisition Date,NaN,Amortized Cost,NaN,Fair Value,NaN,Percentage of Net Assets,NaN,NaN,NaN,NaN,NaN
1,Investment Funds and Vehicles,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,"CIC Flex, LP(10)",NaN,Investment partnership,NaN,Limited partnership units (0.94 units),NaN,NaN,NaN,NaN,9/7/2007,NaN,NaN,-,NaN,NaN,263,(2),NaN,NaN,NaN
3,"Covestia Capital Partners, LP(10)",NaN,Investment partnership,NaN,Limited partnership interest (47.00% interest),NaN,NaN,NaN,NaN,6/17/2008,NaN,NaN,487,NaN,NaN,1862,(2),NaN,NaN,NaN
4,"HCI Equity, LLC(8)(9)(10)",NaN,Investment company,NaN,Member interest (100.00% interest),NaN,NaN,NaN,NaN,4/1/2010,NaN,NaN,-,NaN,NaN,127,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
642,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,10659,NaN,NaN,13534,NaN,NaN,0.26,%
643,Computers and Electronics,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
644,"Everspin Technologies, Inc.(25)",NaN,Designer and manufacturer of computer memory s...,NaN,"First lien senior secured loan ($8,000 par due...",NaN,8.75% (Libor + 7.75%/M),NaN,NaN,6/5/2015,NaN,NaN,7533,NaN,NaN,7840,(5)(21),NaN,NaN,NaN
645,NaN,NaN,NaN,NaN,"Warrant to purchase up to 480,000 shares of Se...",NaN,NaN,NaN,NaN,6/5/2015,NaN,NaN,355,NaN,NaN,355,(5),NaN,NaN,NaN


In [22]:
headers = {
    'User-Agent': 'ARES CAPITAL CORP'
}
# url = 'https://www.sec.gov/Archives/edgar/data/1287750/000128775023000036/arcc-20230630.htm'
# date = 'June 30, 2023'
# url = 'https://www.sec.gov/Archives/edgar/data/1287750/000128775019000013/arccq1-1910q.htm'
# date = 'March 31, 2019'
url = 'https://www.sec.gov/Archives/edgar/data/1287750/000104746914001349/a2218339z10-k.htm'
date = 'December 31, 2013'
# url = 'https://www.sec.gov/Archives/edgar/data/1287750/000128775018000015/arccq1-1810q.htm'
# date = 'March 31, 2018'
# url = 'https://www.sec.gov/Archives/edgar/data/1287750/000110465913080832/a13-19678_110q.htm'
# date = 'September 30, 2013'
# url = 'https://www.sec.gov/Archives/edgar/data/1287750/000128775018000007/arccq4-1710k.htm'
# date = 'December 31, 2017'
url = 'https://www.sec.gov/Archives/edgar/data/1287750/000104746915001240/a2222984z10-k.htm'
date = 'December 31, 2014'
url = 'https://www.sec.gov/Archives/edgar/data/1287750/000104746916010353/a2227293z10-k.htm'
date = 'December 31, 2015'

qtr_date = date

response = requests.get(url, headers=headers)
content = parse_and_trim(response.content, 'HTML')


def extract_tables_manual(soup_content, qtr_date):
    date_regex_pattern1 = r'([A-Za-z]+\s+\d{1,2},\s+\d{4})'
    date_regex_pattern2 = r'\bAs\s+of\s+([A-Za-z]+\s+\d{1,2},\s+\d{4})\b'
    master_table = None
    for tag in soup_content.find_all(text=re.compile(date_regex_pattern2)):
        date_str = re.search(date_regex_pattern1, tag.text)
        find_next = tag.find_next().text
        next_line = tag.next.text
        if re.search('dollar amounts', find_next) or re.search('dollar amounts', next_line):
            print(date_str.group(1))
            if date_str is not None:
                date_str = str(date_str.group(1))
                date_str = unicodedata.normalize('NFKD', date_str)
            if qtr_date.replace(',', '').strip().lower() in date_str.replace(',', '').strip().lower():
                html_table = tag.find_next('table')
                while html_table:
                    new_table = pd.read_html(
                        html_table.prettify(), skiprows=0, flavor='bs4')[0]
                    new_table = new_table.applymap(lambda x: unicodedata.normalize(
                        'NFKD', x.strip().strip(u'\u200b').replace('—', '-')) if type(x) == str else x)
                    new_table = new_table.replace(
                        r'^\s*$', np.nan, regex=True).replace(r'^\s*\$\s*$', np.nan, regex=True)
                    new_table = new_table.dropna(how='all', axis=0)

                    if master_table is None:
                        master_table = new_table
                    else:
                        master_table = pd.concat(
                            [master_table, new_table], ignore_index=True)

                    if date_str.replace(',', '').strip().lower() in 'December 31, 2013'.replace(',', '').strip().lower() or date_str.replace(',', '').strip().lower() in 'December 31, 2014'.replace(',', '').strip().lower():
                        if html_table.find(text=re.compile(r'Food and Beverage', re.IGNORECASE)):
                            break
                    if date_str.replace(',', '').strip().lower() in 'December 31, 2015'.replace(',', '').strip().lower() or date_str.replace(',', '').strip().lower() in 'December 31, 2016'.replace(',', '').strip().lower():
                        if html_table.find(text=re.compile(r'Computers and Electronics', re.IGNORECASE)):
                            break
                    html_table = html_table.find_next('table')

    master_table = master_table.applymap(
        lambda x: x.strip().strip(u'\u200b') if type(x) == str else x)
    master_table = master_table.replace(r'^\s*$', np.nan, regex=True).replace(
        r'^\s*\$\s*$', np.nan, regex=True).replace(r'^\s*\)\s*$', np.nan, regex=True)
    print(master_table.shape)
    return master_table


master_table = extract_tables_manual(content, qtr_date)
master_table.to_csv('example.csv')

master_table

December 31, 2015
December 31, 2014
(647, 20)


,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19
0,Company(1),NaN,Business Description,NaN,Investment,NaN,Interest(6)(12),NaN,Acquisition Date,NaN,Amortized Cost,NaN,Fair Value,NaN,Percentage of Net Assets,NaN,NaN,NaN,NaN,NaN
1,Investment Funds and Vehicles,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,"CIC Flex, LP(10)",NaN,Investment partnership,NaN,Limited partnership units (0.94 units),NaN,NaN,NaN,NaN,9/7/2007,NaN,NaN,-,NaN,NaN,263,(2),NaN,NaN,NaN
3,"Covestia Capital Partners, LP(10)",NaN,Investment partnership,NaN,Limited partnership interest (47.00% interest),NaN,NaN,NaN,NaN,6/17/2008,NaN,NaN,487,NaN,NaN,1862,(2),NaN,NaN,NaN
4,"HCI Equity, LLC(8)(9)(10)",NaN,Investment company,NaN,Member interest (100.00% interest),NaN,NaN,NaN,NaN,4/1/2010,NaN,NaN,-,NaN,NaN,127,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
642,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,10659,NaN,NaN,13534,NaN,NaN,0.26,%
643,Computers and Electronics,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
644,"Everspin Technologies, Inc.(25)",NaN,Designer and manufacturer of computer memory s...,NaN,"First lien senior secured loan ($8,000 par due...",NaN,8.75% (Libor + 7.75%/M),NaN,NaN,6/5/2015,NaN,NaN,7533,NaN,NaN,7840,(5)(21),NaN,NaN,NaN
645,NaN,NaN,NaN,NaN,"Warrant to purchase up to 480,000 shares of Se...",NaN,NaN,NaN,NaN,6/5/2015,NaN,NaN,355,NaN,NaN,355,(5),NaN,NaN,NaN


In [31]:
headers = {
    'User-Agent': 'ARES CAPITAL CORP'
}
# url = 'https://www.sec.gov/Archives/edgar/data/1287750/000128775023000036/arcc-20230630.htm'
# date = 'June 30, 2023'
# url = 'https://www.sec.gov/Archives/edgar/data/1287750/000128775019000013/arccq1-1910q.htm'
# date = 'March 31, 2019'
# url = 'https://www.sec.gov/Archives/edgar/data/1287750/000104746914001349/a2218339z10-k.htm'
# date = 'December 31, 2013'
# url = 'https://www.sec.gov/Archives/edgar/data/1287750/000128775018000015/arccq1-1810q.htm'
# date = 'March 31, 2018'
# url = 'https://www.sec.gov/Archives/edgar/data/1287750/000110465913080832/a13-19678_110q.htm'
# date = 'September 30, 2013'
# url = 'https://www.sec.gov/Archives/edgar/data/1287750/000128775018000007/arccq4-1710k.htm'
# date = 'December 31, 2017'
url = 'https://www.sec.gov/Archives/edgar/data/1287750/000104746915001240/a2222984z10-k.htm'
date = 'December 31, 2014'
url = 'https://www.sec.gov/Archives/edgar/data/1287750/000104746916010353/a2227293z10-k.htm'
date = 'December 31, 2015'

qtr_date = date

response = requests.get(url, headers=headers)
content = parse_and_trim(response.content, 'HTML')


def extract_tables_manual(soup_content, qtr_date):
    date_regex_pattern1 = r'([A-Za-z]+\s+\d{1,2},\s+\d{4})'
    date_regex_pattern2 = r'\bAs\s+of\s+([A-Za-z]+\s+\d{1,2},\s+\d{4})\b'
    master_table = None
    all_tags = soup_content.find_all(True)

    for tag in soup_content.find_all(text=re.compile(date_regex_pattern2)):
        date_str = re.search(date_regex_pattern1, tag.text)
        find_next = tag.find_next().text
        next_line = tag.next.text
        if re.search('dollar amounts', find_next) or re.search('dollar amounts', next_line):
            if re.search('dollar amounts', next_line):
                if date_str is not None:
                    date_str = str(date_str.group(1))
                    date_str = unicodedata.normalize('NFKD', date_str)
                if qtr_date.replace(',', '').strip().lower() in date_str.replace(',', '').strip().lower():
                    html_table = tag.find_next('table')
                    while html_table:
                        new_table = pd.read_html(
                            html_table.prettify(), skiprows=0, flavor='bs4')[0]
                        new_table = new_table.applymap(lambda x: unicodedata.normalize(
                            'NFKD', x.strip().strip(u'\u200b').replace('—', '-')) if type(x) == str else x)
                        new_table = new_table.replace(
                            r'^\s*$', np.nan, regex=True).replace(r'^\s*\$\s*$', np.nan, regex=True)
                        new_table = new_table.dropna(how='all', axis=0)

                        if master_table is None:
                            master_table = new_table
                        else:
                            master_table = pd.concat(
                                [master_table, new_table], ignore_index=True)

                        if date_str.replace(',', '').strip().lower() in 'December 31, 2013'.replace(',', '').strip().lower() or date_str.replace(',', '').strip().lower() in 'December 31, 2014'.replace(',', '').strip().lower():
                            if html_table.find(text=re.compile(r'Food and Beverage', re.IGNORECASE)):
                                break
                        if date_str.replace(',', '').strip().lower() in 'December 31, 2015'.replace(',', '').strip().lower() or date_str.replace(',', '').strip().lower() in 'December 31, 2016'.replace(',', '').strip().lower():
                            if html_table.find(text=re.compile(r'Computers and Electronics', re.IGNORECASE)):
                                break
                        html_table = html_table.find_next('table')
            else:
                if date_str is not None:
                    date_str = str(date_str.group(1))
                    date_str = unicodedata.normalize('NFKD', date_str)
                if qtr_date.replace(',', '').strip().lower() in date_str.replace(',', '').strip().lower():
                    html_table = tag.find_next('table')
                    if master_table is None:
                        master_table = pd.read_html(
                            html_table.prettify(), skiprows=0, flavor='bs4')[0]
                        master_table = master_table.applymap(lambda x: unicodedata.normalize(
                            'NFKD', x.strip().strip(u'\u200b').replace('—', '-')) if type(x) == str else x)
                        master_table = master_table.replace(r'^\s*$', np.nan, regex=True).replace(r'^\s*\$\s*$', np.nan,
                                                                                                  regex=True)
                        master_table = master_table.dropna(how='all', axis=0)
                    else:
                        new_table = pd.read_html(
                            html_table.prettify(), skiprows=0, flavor='bs4')[0]
                        new_table = new_table.applymap(lambda x: unicodedata.normalize(
                            'NFKD', x.strip().strip(u'\u200b').replace('—', '-')) if type(x) == str else x)
                        new_table = new_table.replace(r'^\s*$', np.nan, regex=True).replace(r'^\s*\$\s*$', np.nan,
                                                                                            regex=True)
                        new_table = new_table.dropna(how='all', axis=0)
                        # print('head')
                        # print(new_table.head()) # text
                        master_table = master_table.append(
                            new_table.dropna(how='all', axis=0).reset_index(
                                drop=True).drop(index=0),
                            ignore_index=True)

    master_table = master_table.applymap(
        lambda x: x.strip().strip(u'\u200b') if type(x) == str else x)
    master_table = master_table.replace(r'^\s*$', np.nan, regex=True).replace(
        r'^\s*\$\s*$', np.nan, regex=True).replace(r'^\s*\)\s*$', np.nan, regex=True)
    print(master_table.shape)
    return master_table


master_table = extract_tables_manual(content, qtr_date)
master_table.to_csv('example.csv')

master_table

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19
0,Company(1),NaN,Business Description,NaN,Investment,NaN,Interest(6)(12),NaN,Acquisition Date,NaN,Amortized Cost,NaN,Fair Value,NaN,Percentage of Net Assets,NaN,NaN,NaN,NaN,NaN
1,Investment Funds and Vehicles,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,"CIC Flex, LP(10)",NaN,Investment partnership,NaN,Limited partnership units (0.94 units),NaN,NaN,NaN,NaN,9/7/2007,NaN,NaN,-,NaN,NaN,263,(2),NaN,NaN,NaN
3,"Covestia Capital Partners, LP(10)",NaN,Investment partnership,NaN,Limited partnership interest (47.00% interest),NaN,NaN,NaN,NaN,6/17/2008,NaN,NaN,487,NaN,NaN,1862,(2),NaN,NaN,NaN
4,"HCI Equity, LLC(8)(9)(10)",NaN,Investment company,NaN,Member interest (100.00% interest),NaN,NaN,NaN,NaN,4/1/2010,NaN,NaN,-,NaN,NaN,127,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
642,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,10659,NaN,NaN,13534,NaN,NaN,0.26,%
643,Computers and Electronics,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
644,"Everspin Technologies, Inc.(25)",NaN,Designer and manufacturer of computer memory s...,NaN,"First lien senior secured loan ($8,000 par due...",NaN,8.75% (Libor + 7.75%/M),NaN,NaN,6/5/2015,NaN,NaN,7533,NaN,NaN,7840,(5)(21),NaN,NaN,NaN
645,NaN,NaN,NaN,NaN,"Warrant to purchase up to 480,000 shares of Se...",NaN,NaN,NaN,NaN,6/5/2015,NaN,NaN,355,NaN,NaN,355,(5),NaN,NaN,NaN
